In [27]:
from __future__ import print_function
from __future__ import division

import os
import sys
import argparse
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, transforms

from imagenet_dataset import ImageNetDataset
from torch.utils.data import DataLoader

In [22]:
sys.argv = ['uniform_testing.ipynb', 
            '--no-cuda', 'False',
            '--seed', '5', 
            '--model-path', './checkpoints',
            '--test-batch-size', '50',
            '--target-count', '20',
            '--target-type', '0',
            '--target-label', '-1'
            ]
parser = argparse.ArgumentParser(description='Black-box Adversarial Attack')
parser.add_argument('--no-cuda', default=False,
                    help='disables CUDA training')
parser.add_argument('--seed', type=int, default=0, metavar='S',
                    help='random seed (default: 0)')
parser.add_argument('--model-path', default='../../checkpoints',
                    help='directory of model for saving checkpoint')
parser.add_argument('--test-batch-size', type=int, default=50, metavar='N',
                    help='input batch size for testing (default: 50)')
parser.add_argument('--target-count', type=int, default=20, metavar='N',
                    help='the amount of targets(default: 20)')
parser.add_argument('--target-type', type=int, default=0, metavar='N',
                    help='the method of choosing target label.\n0: ini_label + 1;\n1: all the other labels')
parser.add_argument('--target-label', type=int, default=-1, metavar='N',
                    help='target label, default: -1, which is (ini_label + 1) % 10')
args = parser.parse_args()
print(args)

Namespace(no_cuda='False', seed=5, model_path='../../checkpoints', test_batch_size=50, target_count=20, target_type=0, target_label=-1)


In [23]:
# specify optimization related parameters
LR = 0.05  # learning rate
EPOCHS = 20  # total optimization epochs
NB_SAMPLE = 1000  # number of samples for adjusting lambda
MINI_BATCH = NB_SAMPLE // args.test_batch_size  # number of batches
INIT_COST = 1e-3  # initial weight of lambda

ATTACK_SUCC_THRESHOLD = 0.99  # attack success threshold
PATIENCE = 5  # patience for adjusting lambda, number of mini batches
COST_MULTIPLIER = 2  # multiplier for auto-control of weight (COST)
COST_MULTIPLIER_UP = COST_MULTIPLIER
COST_MULTIPLIER_DOWN = 10 ** 0.5  # changed from 2**1.5 to 10**0.5

EARLY_STOP_THRESHOLD = 1.0  # loss threshold for early stop
EARLY_STOP_PATIENCE = 5 * PATIENCE  # patience for early stop
EPSILON = 1e-8

In [28]:
def data_factory(dataset_type = 'Cifar10', no_cuda = False, batch_size = 1):
    use_cuda = not no_cuda and torch.cuda.is_available()
    if dataset_type == 'Cifar10':
        kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
        dataset = torchvision.datasets.CIFAR10(root='./datasets', train=False, download=True, transform=transforms.Compose([transforms.ToTensor()]))
        return DataLoader(testset, batch_size=batch_size, shuffle=False, **kwargs)
    elif data_factory == 'ImageNet':
        im_size = 299
        mean_arr = (0.5, 0.5, 0.5)
        stddev_arr = (0.5, 0.5, 0.5)
        dataset = ImageNetDataset(
            image_dir='./datasets/imagenet1000',
            label_filepath="./datasets/imagenet_label.txt",
            transform=transforms.Compose([
                transforms.CenterCrop(im_size),
                transforms.ToTensor(),
                transforms.Normalize(mean_arr, stddev_arr)
            ]),
        )
        return DataLoader(dataset, batch_size=batch_size, shuffle=False)
 

def test_attack_success_rate(config, target_model):
    # sourcery skip: last-if-guard, use-fstring-for-concatenation
    """This is the unified model for testing attack success rate for adversarial success under a certain configuration.

    Args:
        config (dictionary): include the attack type(black or white, targeted or untargeted) and other necessary information
        target_model: the model to be attacked
        data_loader: batch_size = 1
    """
    dataloader = data_factory(config.dataset_type)
    if config.target_type == 'Untargeted' and config.data == 'Cifar10':
        for i, data in enumerate(dataloader):
            print(f'=======Attacking Target {str(i)}=======')
            image = data['image']
            init_label = data['label']
            classified_label = target_model(image)
            if classified_label != init_label:
                print('Classification error: {init_label} != {classified_label}. Jump to next image.')
                continue
            
            mask, pattern = config.attack(target_model, image, init_label)  # untargeted attack
            
            # store mask, fusion, pattern; attacked_target
            im = Image.fromarray((mask * 255).astype(np.uint8))
            im.save(config.pert_path + '/mask_' +  str(i) + '.png')
            im = Image.fromarray((pattern * 255).astype(np.uint8))
            im.save(config.pert_path + '/pattern_' + str(i) + '.png')
            im = Image.fromarray((mask.reshape([32,32,1]) * pattern * 255).astype(np.uint8))
            im.save(config.pert_path + '/fusion_' + str(i) + '.png')

            img = image.detach().permute(1,2,0).cpu().numpy()
            im = Image.fromarray(((mask.reshape([32,32,1]) * pattern + img) * 255).astype(np.uint8))
            im.save(config.adv_path + '/adv_' + str(i) + '.png')
            

In [ ]:
def attack_factory(attack_algorithm):
    mapping = {'B3D': B3D_attack,
               'greedyfool': greedyfool_attack, 
               'cornersearch': cornersearch_attack, 
               'PGD_attack': PGD_attack, 
               'homotopy': homotopy_attack, 
               'perturbation': perturbation_attack, 
               'imperceptible': imperceptible_attack
               }
    return mapping[attack_algorithm]

def B3D_attack():
    pass

def greedyfool_attack():
    pass

def cornersearch_attack():
    pass

def PGD_attack():
    pass

def homotopy_attack():
    pass

def perturbation_attack():
    pass

def imperceptible_attack():
    pass